In [179]:
import pandas as pd
import numpy as np
train_titanic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML python/Competition_kaggle/datasets/train.csv")
test_titanic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML python/Competition_kaggle/datasets/test.csv")
train_titanic.head()
test_titanic.shape

(4277, 13)

In [180]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [181]:
test_titanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


>+ I shuffle my data with the desire to improve the model's learning ability

In [182]:
from sklearn.utils import shuffle
train_titanic = shuffle(train_titanic)
test_titanic = shuffle(test_titanic)

In [183]:
train_titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 6645 to 5408
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 959.3+ KB


In [184]:
train_titanic.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

### **Features scalling data**

In [185]:
col_names = "RoomService", "FoodCourt","ShoppingMall","Spa","VRDeck"
roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix = [
    train_titanic.columns.get_loc(c) for c in col_names] # get the column indices
print(roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix)

7 8 9 10 11


In [186]:
import math
from sklearn.base import BaseEstimator, TransformerMixin
attribs_spending_money = [7,8,9,10,11]
# roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix = 7,8,9,10,11
class CombineSpendingMoneyOfPerson(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        array = np.zeros(len(X)) # mang mot cot toan so 0
        for ele in range(len(X)):
            for i in attribs_spending_money:
                if ~math.isnan(X[ele][i]):
                    array[ele] += (X[ele][i])
        return np.c_[X,array]

# function for add feature ["spen_money_of_person"]
def feature_scalling_dataset(train, test, drop=None):
    attr_adder = CombineSpendingMoneyOfPerson()
    # Dataset with added columns ["spending_money_of_person"]
    train_titanic_extra_attribs = attr_adder.transform(train.values)
    test_titanic_extra_attribs = attr_adder.transform(test.values)
    train_titanic_extra_attribs = pd.DataFrame(train_titanic_extra_attribs,
                            columns=list(train) + ["spending_money_of_person"],
                            index= train.index)
    test_titanic_extra_attribs = pd.DataFrame(test_titanic_extra_attribs,
                            columns=list(test) + ["spending_money_of_person"],
                            index= test.index)
    return train_titanic_extra_attribs, test_titanic_extra_attribs
train_titanic_1, test_titanic_1 = feature_scalling_dataset(train_titanic, test_titanic)
train_titanic_1.isnull().sum()

PassengerId                   0
HomePlanet                  201
CryoSleep                   217
Cabin                       199
Destination                 182
Age                         179
VIP                         203
RoomService                 181
FoodCourt                   183
ShoppingMall                208
Spa                         183
VRDeck                      188
Name                        200
Transported                   0
spending_money_of_person    908
dtype: int64

In [187]:
# Split columns ["Cabin"] to ["Deck","Side"]
def deck_side_encode(data):
    split_cabin = pd.DataFrame(list(map(lambda x: str(x).split("/"), data.Cabin)))
    new_data = data.copy()
    new_data["Side"] = split_cabin[2]
    new_data["Deck"] = split_cabin[0]
    new_data = new_data.drop("Cabin", axis=1)
    return new_data

In [188]:
# for handle categories feature
from sklearn.preprocessing import LabelEncoder

def encode_labels(datasets):
  label_encoder = LabelEncoder()
  for column in datasets:
        dtype = datasets[column].dtype
        if dtype not in ['int64','float64']:
            datasets[column] = label_encoder.fit_transform(datasets[column])
  return datasets


In [189]:
train_ok = deck_side_encode(train_titanic)
train_ok = encode_labels(train_ok)
train_ok.head()


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Side,Deck
6645,6645,1,0,2,27.0,0,0.0,2897.0,659.0,485.0,11.0,5161,1,1,6
8565,8565,0,0,2,11.0,0,0.0,0.0,0.0,0.0,0.0,2657,0,0,3
4267,4267,1,1,0,20.0,0,0.0,0.0,0.0,0.0,0.0,748,1,0,6
1433,1433,1,0,2,40.0,0,1006.0,NaN,0.0,284.0,329.0,518,1,1,3
1588,1588,0,0,0,7.0,0,0.0,0.0,0.0,0.0,0.0,7953,0,0,5


In [190]:
attribs_num = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","spending_money_of_person"]
attribs_cat = ["HomePlanet", "Destination", "Cabin"]
attribs_cat_bool = ["CryoSleep", "VIP"]

### **Handle Data**

In [191]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

def prepare_data(train, test, drop=None):
    train, test = feature_scalling_dataset(train, test)
    # delete columns Name and PassengerId
    train = train.drop(["Name", "PassengerId"], axis=1)
    test = test.drop(["Name", "PassengerId"], axis=1)
    # missing value
    imputer = SimpleImputer(strategy="mean")
    imputer_cat_bool = SimpleImputer(strategy="most_frequent")

    train[attribs_num] = imputer.fit_transform(train[attribs_num])
    test[attribs_num] = imputer.fit_transform(test[attribs_num])

    train[attribs_cat_bool] = imputer_cat_bool.fit_transform(train[attribs_cat_bool])
    test[attribs_cat_bool] = imputer_cat_bool.fit_transform(test[attribs_cat_bool])

    train[attribs_cat] = imputer_cat_bool.fit_transform(train[attribs_cat])
    test[attribs_cat] = imputer_cat_bool.fit_transform(test[attribs_cat])
    
    # implement LabelEncoder()
    train_encoded = encode_labels(train)
    test_encoded = encode_labels(test)
    # test_encoded_titan_encoder["Transported"] = test_encoded_titan_encoder["Transported"].astype(int)
    return train_encoded, test_encoded 

In [192]:
train_prep, test_prep = prepare_data(train_titanic, test_titanic)

In [193]:
train_prep.isnull().sum()

HomePlanet                  0
CryoSleep                   0
Cabin                       0
Destination                 0
Age                         0
VIP                         0
RoomService                 0
FoodCourt                   0
ShoppingMall                0
Spa                         0
VRDeck                      0
Transported                 0
spending_money_of_person    0
dtype: int64

In [194]:
train_prep.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,spending_money_of_person
6645,1,0,1915,2,27.0,0,0.0,2897.000000,659.0,485.0,11.0,1,4052.000000
8565,0,0,5242,2,11.0,0,0.0,0.000000,0.0,0.0,0.0,0,0.000000
4267,1,1,233,0,20.0,0,0.0,0.000000,0.0,0.0,0.0,1,0.000000
1433,1,0,548,2,40.0,0,1006.0,458.077203,0.0,284.0,329.0,1,1484.601541
1588,0,0,5471,0,7.0,0,0.0,0.000000,0.0,0.0,0.0,0,0.000000


### **Create and evaluate models**

In [195]:
X_train = np.array(train_prep.drop("Transported", axis=1))
y_train = np.array(train_prep.Transported)
X_test = np.array(test_prep)
X_train.shape, X_test.shape, y_train.shape

((8693, 12), (4277, 12), (8693,))

In [196]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(MinMaxScaler(feature_range=(0,10)), LogisticRegression(random_state=42, solver='lbfgs'))
cross_val_score(pipeline, X_train, y_train, cv=3, scoring="accuracy").mean()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.7836197524405716

#### Using DecisonTreeClassification


In [197]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {
    'criterion': ["gini","log_loss", "entropy"],
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth':randint(low=2, high=20),
    "min_samples_split":randint(low=1, high = 5),
}

tree_classifier = DecisionTreeClassifier(random_state=42)
tree_grid_search = RandomizedSearchCV(tree_classifier, param_distributions=param_grid,
                                            n_iter=10, cv= 5, scoring="accuracy", random_state=42, verbose=1)

In [198]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_tf = scaler.fit_transform(X_train)
X_test_tf = scaler.fit_transform(X_test)
tree_grid_search.fit(X_train_tf, y_train)
# tree_pipeline = make_pipeline(StandardScaler(), tree_grid_search)
X_test_tf.shape

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 254, in fit
    % self.min_samples_split
ValueError: min_samples_split must be an integer greater than 1 or a float i

(4277, 12)

In [199]:
tree_grid_search.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=11, max_leaf_nodes=93,
                       min_samples_split=4, random_state=42)

In [200]:
tree_grid_search.best_score_

0.7846563405949347

In [201]:
finish_model_decison_tree_classifier = tree_grid_search.best_estimator_

### Evalue model with f1_score

In [202]:
from sklearn.metrics import f1_score
y_pred = finish_model_decison_tree_classifier.predict(X_train_tf)
f1_score(y_train, y_pred)

0.8227751820058676

In [203]:
from sklearn.metrics import classification_report
evalue = classification_report(y_train, y_pred)
print(evalue)

              precision    recall  f1-score   support

           0       0.85      0.76      0.80      4315
           1       0.78      0.86      0.82      4378

    accuracy                           0.81      8693
   macro avg       0.82      0.81      0.81      8693
weighted avg       0.82      0.81      0.81      8693



#### **GradientBoostingClassifier**

In [204]:
from sklearn.ensemble import GradientBoostingClassifier
boosting_param_grid = {
    'loss':["deviance", "exponential"],
    'learning_rate':[0.03, 0.07, 0.1, 0.2],
    'n_estimators': [101, 151, 201, 301, 501, 1000],
    'criterion':["friedman_mse", "squared_error", "mse"],
}

gradient_boosting_classifier = GradientBoostingClassifier()
boosting_grid = RandomizedSearchCV(gradient_boosting_classifier, param_distributions=boosting_param_grid,
                                    n_iter=10, cv= 5, scoring="accuracy", random_state=42, verbose=1)

In [205]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(0,5))
pipeline_boosting = make_pipeline(min_max_scaler, boosting_grid)
pipeline_boosting.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/di

Pipeline(steps=[('minmaxscaler', MinMaxScaler(feature_range=(0, 5))),
                ('randomizedsearchcv',
                 RandomizedSearchCV(cv=5,
                                    estimator=GradientBoostingClassifier(),
                                    param_distributions={'criterion': ['friedman_mse',
                                                                       'squared_error',
                                                                       'mse'],
                                                         'learning_rate': [0.03,
                                                                           0.07,
                                                                           0.1,
                                                                           0.2],
                                                         'loss': ['deviance',
                                                                  'exponential'],
                                    

In [206]:
boosting_grid.best_estimator_

GradientBoostingClassifier(criterion='mse', learning_rate=0.07,
                           loss='exponential', n_estimators=301)

In [207]:
boosting_grid.best_score_

0.804212372889992

In [208]:
boosting_model = boosting_grid.best_estimator_

In [209]:
y_pred_boosting = boosting_model.predict(X_train_tf)
error = classification_report(y_train, y_pred_boosting)
print(error)

              precision    recall  f1-score   support

           0       0.86      0.48      0.62      4315
           1       0.64      0.92      0.76      4378

    accuracy                           0.70      8693
   macro avg       0.75      0.70      0.69      8693
weighted avg       0.75      0.70      0.69      8693



In [210]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import NuSVC
from sklearn.metrics import accuracy_score

import xgboost as xgb

estimators = [
    ('xgb', xgb.XGBClassifier(max_depth=3, n_estimators=91, subsample=0.65, colsample_bytree=0.65, eval_metric='rmse')),
    ('tree', DecisionTreeClassifier(criterion='entropy', max_depth=16, max_leaf_nodes=63,min_samples_split=3, random_state=42)),
    ('nusvc', make_pipeline(StandardScaler(), NuSVC())),
    ('knn', KNeighborsClassifier(n_neighbors=38))
]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [211]:
clf.fit(X_train_tf, y_train)

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(colsample_bytree=0.65,
                                              eval_metric='rmse',
                                              n_estimators=91,
                                              subsample=0.65)),
                               ('tree',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=16,
                                                       max_leaf_nodes=63,
                                                       min_samples_split=3,
                                                       random_state=42)),
                               ('nusvc',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('nusvc', NuSVC())])),
                               ('knn', KNei

In [212]:
y_pred_stacking = clf.predict(X_train_tf)

In [213]:
err_stacking = classification_report(y_train, y_pred_stacking)
print(err_stacking)

              precision    recall  f1-score   support

           0       0.84      0.79      0.81      4315
           1       0.81      0.85      0.83      4378

    accuracy                           0.82      8693
   macro avg       0.82      0.82      0.82      8693
weighted avg       0.82      0.82      0.82      8693



In [214]:
test_titanic_ = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML python/Competition_kaggle/datasets/test.csv")
test_titanic_.shape

(4277, 13)

In [215]:
y1 = boosting_model.predict(X_test_tf)
y2 = clf.predict(X_test_tf)

In [216]:
result = pd.DataFrame(y1, columns=["Transported"])
result.shape

(4277, 1)

In [217]:
test_id = test_titanic_["PassengerId"]
submission = pd.DataFrame({
    "PassengerId":test_id,
    "Transported":y1.astype(bool),
})
display(submission.head(10))

,PassengerId,Transported
0,0013_01,True
1,0018_01,True
2,0019_01,True
3,0021_01,True
4,0023_01,True
5,0027_01,True
6,0029_01,True
7,0032_01,True
8,0032_02,False
9,0033_01,True


In [218]:
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/ML python/Competition_kaggle/submission.csv", index=False)

In [219]:
test_id = test_titanic_["PassengerId"]
submission1 = pd.DataFrame({
    "PassengerId":test_id,
    "Transported":y2.astype(bool),
})
display(submission.head(10))

,PassengerId,Transported
0,0013_01,True
1,0018_01,True
2,0019_01,True
3,0021_01,True
4,0023_01,True
5,0027_01,True
6,0029_01,True
7,0032_01,True
8,0032_02,False
9,0033_01,True


In [220]:
submission1.to_csv("/content/drive/MyDrive/Colab Notebooks/ML python/Competition_kaggle/submission1.csv", index=False)

In [221]:
from sklearn.ensemble import RandomForestClassifier
forest_param_grid = {
    'n_estimators':[100,200,300,400,700,1000],
    'criterion':["gini", "entropy", "log_loss"],
    'max_depth':[3,4,5,6,7,10],
}
forest_classifier = RandomForestClassifier(random_state=42)
forest_grid = RandomizedSearchCV(forest_classifier, param_distributions=forest_param_grid,
                                 n_iter=25, cv = 10, verbose=2, random_state=42)

In [222]:
forest_grid.fit(X_train_tf, y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.3s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.3s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   0.4s
[CV] END ......criterion=gini, max_depth=4, n_estimators=700; total time=   3.9s
[CV] END ......criterion=gini, max_depth=4, n_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/par

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
                   n_iter=25,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [3, 4, 5, 6, 7, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         700, 1000]},
                   random_state=42, verbose=2)

In [223]:
forest_grid.best_estimator_

RandomForestClassifier(max_depth=10, random_state=42)

In [224]:
model_forest = forest_grid.best_estimator_

In [225]:
y_pred3 = model_forest.predict(X_train_tf)
forest_error = classification_report(y_train, y_pred3)
print(forest_error)

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      4315
           1       0.84      0.87      0.86      4378

    accuracy                           0.85      8693
   macro avg       0.85      0.85      0.85      8693
weighted avg       0.85      0.85      0.85      8693



In [226]:
y3 = model_forest.predict(X_test_tf)
test_id = test_titanic_["PassengerId"]
submission1 = pd.DataFrame({
    "PassengerId":test_id,
    "Transported":y3.astype(bool),
})
display(submission.head())

submission.to_csv("/content/drive/MyDrive/Colab Notebooks/ML python/Competition_kaggle/submission3.csv", index=False)

,PassengerId,Transported
0,0013_01,True
1,0018_01,True
2,0019_01,True
3,0021_01,True
4,0023_01,True
